In [1]:
## import
from conll_iterator import ConllIterator

In [2]:
## Creates an object (ConllIterator) that iterates over sentences of a ConLL file.

corpus = "sample/sample_corpus.conllu"
# dictionary with features and their position in the ConLL lines.
# format: {<feature, str>: <position, int>}  
idx_dict = {"id": 0, "surface": 1, "lemma": 2, "pos": 3}
sentences = ConllIterator(corpus, idx_dict, codec='utf8')
# set_itermode changes the behaviour of the iterator.
sentences.set_itermode('sent', keys=["surface", "lemma", "pos"], ignore_compound=True)

In [ ]:
## Prints first 100 elements yelded by the iterator
sentences.sample()